In [138]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import util
from openai import OpenAI

In [59]:
with open('data.json', 'r') as file:
    resume = json.load(file)
def create_documents_dynamic(resume):
    documents = []

    for key, value in resume.items():
        if isinstance(value, dict):  # Handle nested dictionary (e.g., contact information)
            section = f"{key.replace('_', ' ').title()}: " + ", ".join(
                f"{sub_key.title()}: {sub_value}" for sub_key, sub_value in value.items()
            )
            documents.append(section)

        elif isinstance(value, list):  # Handle lists (e.g., education, projects)
            for item in value:
                if isinstance(item, dict):  # Handle list of dictionaries
                    section = f"{key.replace('_', ' ').title()}:\n" + "\n".join(
                        f"{sub_key.replace('_', ' ').title()}: {sub_value}" for sub_key, sub_value in item.items()
                    )
                    documents.append(section)
                else:  # Handle plain list
                    documents.append(f"{key.replace('_', ' ').title()}: {', '.join(value)}")
        
        elif isinstance(value, str):  # Handle plain string fields
            documents.append(f"{key.replace('_', ' ').title()}: {value}")
        
        else:  # Handle unexpected formats
            documents.append(f"{key.replace('_', ' ').title()}: {value}")

    return documents

# Example Usage
documents = create_documents_dynamic(resume)
print(documents)

['Summary: A dedicated Electronics and Telecommunication professional with a strong background in machine learning, web development, and cloud platforms. Proven ability to lead projects, solve complex problems, and continuously learn new technologies to drive innovation and efficiency.', 'Contact: Name: Aneesh Patne, Email: aneeshpatne12@gmail.com, Linkedin: https://www.linkedin.com/in/aneeshpatne, Github: https://github.com/aneeshpatne, Leetcode: https://leetcode.com/aneeshpatne', 'Education:\nDegree: M.Tech in Electronics and Telecommunication\nInstitution: Veermata Jijabai Technological Institute\nLocation: Mumbai, Maharashtra\nDuration: 2023 - 2025\nDetails: Specialized in Machine Learning and Signal Processing. Relevant coursework includes Advanced Algorithms, Neural Networks, and Communication Systems.', 'Education:\nDegree: B.Tech in Electronics and Telecommunication\nInstitution: Thakur College of Engineering and Technology\nLocation: Mumbai, Maharashtra\nDuration: 2019 - 2023\

In [106]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2') 

In [107]:
document_embeddings = embedding_model.encode(documents, convert_to_tensor=False, show_progress_bar=True)

Batches: 100%|██████████| 1/1 [00:00<00:00, 18.01it/s]


In [108]:
document_embeddings = np.array(document_embeddings).astype('float32')

In [ ]:
res = faiss.StandardGpuResources()
index_flat = faiss.IndexFlatL2(document_embeddings.shape[1])
gpu_index = faiss.index_cpu_to_gpu(res, 0, index_flat)
gpu_index.add(document_embeddings)


In [110]:
index = faiss.IndexFlatL2(document_embeddings.shape[1])  # Using L2 distance
index.add(document_embeddings)

In [111]:
faiss.write_index(index, 'resume_index.faiss')

In [112]:
with open('documents.txt', 'w') as f:
    for doc in documents:
        f.write(doc.replace('\n', ' ') + '\n')

In [113]:
def retrieve_relevant_documents(query, top_k=3):

    query_embedding = embedding_model.encode([query], convert_to_tensor=False)
    query_embedding = np.array(query_embedding).astype('float32')
    

    index = faiss.read_index('resume_index.faiss')
    
  
    distances, indices = index.search(query_embedding, top_k)
    

    with open('documents.txt', 'r') as f:
        all_documents = f.readlines()
    

    relevant_docs = [all_documents[idx].strip() for idx in indices[0]]
    return relevant_docs


In [114]:
model_path = "D:/Llama-3.2-1B-Instruct"  # Replace with your actual path


In [115]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
).to("cpu")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [53]:
def generate_response(query):
    # Retrieve relevant documents
    relevant_docs = retrieve_relevant_documents(query)
    
    if not relevant_docs:  # No relevant documents found
        return "I'm sorry, this information is not available in the resume."
    
    # Concatenate the documents
    context = "\n".join(relevant_docs)
    
    # Compute similarity
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    context_embedding = embedding_model.encode(context, convert_to_tensor=True)
    similarity = util.cos_sim(query_embedding, context_embedding).item()
    
    if similarity < 0.3:  # Low similarity threshold
        return f"I'm sorry, this information is not available in the resume. (Similarity: {similarity:.2f})"
    
    # Construct prompt
    prompt = (
        "You are an AI assistant answering questions about a person's resume. "
        "Respond only with information present in the context below. If the answer "
        "cannot be found, respond with 'I'm sorry, this information is not available in the resume, while explaining what went wrong in their question'\n\n"
        f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    )
    
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True)
    
    # Generate the response
    outputs = model.generate(
        inputs.input_ids,
        max_length=500,
        do_sample=True,
        temperature=0.2
    )
    
    # Decode the response
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the answer part
    answer = answer.split('Answer:')[-1].strip()
    
    return answer

# Example query
query = "Socio-Economic Impact of Pollution on Life Expectancy"
print(generate_response(query))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I'm sorry, this information is not available in the resume, while explaining what went wrong in their question. The question seems to be asking for information about the project, but it does not provide enough context or details about the project. The project description provided does not mention the socio-economic impact of pollution on life expectancy. The technologies used in the project are also not mentioned in the question.


In [69]:
def generate_response(query):
    # Retrieve relevant documents
    relevant_docs = retrieve_relevant_documents(query)
    
    # Concatenate the documents
    context = "\n".join(relevant_docs)
    print(context)
    # Create the prompt
    prompt = (
    "You are an AI assistant answering questions about a person's resume. "
    "Only use the information provided in the context below to answer the question. "
    "If the answer cannot be found in the context, respond with 'I'm sorry, this information is not available.'\n\n"
    f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
)

    
    # Tokenize the input
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    # Generate the response
    outputs = model.generate(inputs, max_length=500, do_sample=True, temperature=0.2)
    
    # Decode the response
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the answer part
    answer = answer.split('Answer:')[-1].strip()
    return answer
query = "Explain me about the project Socio-Economic Impact of Pollution on Life Expectancy"
print(generate_response(query))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Projects: Title: Socio-Economic Impact of Pollution on Life Expectancy Description: A machine learning project analyzing the impact of air pollution on life expectancy, integrating multiple socio-economic parameters. Technologies Used: ['Machine Learning', 'Google Vertex AI', 'SvelteKit', 'Express'] Duration: January 2024 - Present Tags: ['Machine Learning', 'Data Analysis', 'Sustainability']
Projects: Title: IoT and ML based Weather Prediction System Description: Designed and implemented an embedded system for automating household tasks, enhancing efficiency and reliability. Technologies Used: ['C++', 'Arduino', 'IoT'] Duration: September 2022 - December 2022 Tags: ['Embedded Systems', 'IoT', 'Automation']
Contact: Name: Aneesh Patne, Email: aneeshpatne12@gmail.com, Linkedin: https://www.linkedin.com/in/aneeshpatne, Github: https://github.com/aneeshpatne, Leetcode: https://leetcode.com/aneeshpatne
Unfortunately, I'm sorry, this information is not available.


In [169]:
def compute_similarity_per_segment(query, documents):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    max_similarity = 0
    best_doc = None
    for doc in documents:
        doc_embedding = embedding_model.encode(doc, convert_to_tensor=True)
        similarity = util.cos_sim(query_embedding, doc_embedding).item()
        if similarity > max_similarity:
            max_similarity = similarity
            best_doc = doc
    
    return max_similarity, best_doc


In [137]:
def generate_response(query):
    # Retrieve relevant documents
    relevant_docs = retrieve_relevant_documents(query)
    print("Retrieved documents:", relevant_docs)
    
    if not relevant_docs:
        return "I'm sorry, this information is not available in the resume."
    
    # Compute similarity per segment
    similarity, best_doc = compute_similarity_per_segment(query, relevant_docs)
    print(f"Best document: {best_doc}")
    print(f"Similarity score: {similarity}")
    

    
    # Use the most relevant document as context
    context = relevant_docs
    prompt = (
        "You are an 'Chotu' Who is Aneesh's assistant answering questions about a person's resume. "
        "Use only the provided context to answer the question. "
        "Do not generate any information that is not explicitly stated in the context. "
        "If the information is not available, respond with: 'I'm sorry, this information is not available.'\n\n"
        f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    )
        
    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True)
    outputs = model.generate(inputs.input_ids, max_length=500, do_sample=True, temperature=0.7)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the answer part
    answer = answer.split('Answer:')[-1].strip()
    return answer

# Example query
query = "How to contact Aneesh ?"
print(generate_response(query))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Retrieved documents: ['Contact: Name: Aneesh Patne, Email: aneeshpatne12@gmail.com, Linkedin: https://www.linkedin.com/in/aneeshpatne, Github: https://github.com/aneeshpatne, Leetcode: https://leetcode.com/aneeshpatne', 'Certifications: Title: Full Stack Web Development Issuing Organization: Udemy Date Awarded: April 2022 Credential Id: FSWD67890', 'Education: Degree: B.Tech in Electronics and Telecommunication Institution: Thakur College of Engineering and Technology Location: Mumbai, Maharashtra Duration: 2019 - 2023 Details: Graduated with First Class Honors. Engaged in multiple projects focusing on embedded systems and wireless communication.']
['Contact: Name: Aneesh Patne, Email: aneeshpatne12@gmail.com, Linkedin: https://www.linkedin.com/in/aneeshpatne, Github: https://github.com/aneeshpatne, Leetcode: https://leetcode.com/aneeshpatne', 'Certifications: Title: Full Stack Web Development Issuing Organization: Udemy Date Awarded: April 2022 Credential Id: FSWD67890', 'Education: D

In [77]:
# Initialize memory
chat_history = []

def generate_response(query):
    global chat_history  # Use global chat memory
    
    # Retrieve relevant documents
    relevant_docs = retrieve_relevant_documents(query)
    print("Retrieved documents:", relevant_docs)
    
    if not relevant_docs:
        return "I'm sorry, this information is not available in the resume."
    print("Relevant Docs Found")
    # Compute similarity per segment
    similarity, best_doc = compute_similarity_per_segment(query, relevant_docs)
    print("Best Document Found")
    
    if similarity < 0.4:  # Adjust threshold
        return "I'm sorry, this information is not available in the resume."

    # Use the most relevant document as context
    context = best_doc
    
    # Incorporate chat memory
    history = "\n".join([f"User: {entry['user']}\nAI: {entry['ai']}" for entry in chat_history])
    prompt = (
        "You are an AI assistant answering questions about a person's resume. "
        "Continue the conversation based on the context and prior history provided below. "
        "Answer strictly using the provided information. If the answer cannot be found, respond with "
        "'I'm sorry, this information is not available.'\n\n"
        f"Context:\n{context}\n\nConversation History:\n{history}\n\nUser: {query}\nAI:"
    )
    
    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True)
    outputs = model.generate(inputs.input_ids, max_length=500, do_sample=True, temperature=0.7)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the answer part
    answer = answer.split('Answer:')[-1].strip()
    
    # Update memory
    chat_history.append({"user": query, "ai": answer})
    
    # Handle memory token limit
    chat_history = trim_history(chat_history)
    
    return answer

def trim_history(chat_history, max_tokens=1024):
    # Trim conversation history to fit within the token limit
    history_text = "\n".join([f"User: {entry['user']}\nAI: {entry['ai']}" for entry in chat_history])
    tokens = tokenizer(history_text, return_tensors='pt')['input_ids'].size(1)
    
    while tokens > max_tokens and chat_history:
        chat_history.pop(0)  # Remove the oldest interaction
        history_text = "\n".join([f"User: {entry['user']}\nAI: {entry['ai']}" for entry in chat_history])
        tokens = tokenizer(history_text, return_tensors='pt')['input_ids'].size(1)
    
    return chat_history

# Example query
while True:
    query = input("User: ")
    if query.lower() in ["exit", "quit"]:
        print("Exiting chat. Goodbye!")
        break
    
    response = generate_response(query)
    print(f"AI: {response}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Retrieved documents: ["Projects: Title: Socio-Economic Impact of Pollution on Life Expectancy Description: A machine learning project analyzing the impact of air pollution on life expectancy, integrating multiple socio-economic parameters. Technologies Used: ['Machine Learning', 'Google Vertex AI', 'SvelteKit', 'Express'] Duration: January 2024 - Present Tags: ['Machine Learning', 'Data Analysis', 'Sustainability']", 'Education: Degree: B.Tech in Electronics and Telecommunication Institution: Thakur College of Engineering and Technology Location: Mumbai, Maharashtra Duration: 2019 - 2023 Details: Graduated with First Class Honors. Engaged in multiple projects focusing on embedded systems and wireless communication.', "Projects: Title: IoT and ML based Weather Prediction System Description: Designed and implemented an embedded system for automating household tasks, enhancing efficiency and reliability. Technologies Used: ['C++', 'Arduino', 'IoT'] Duration: September 2022 - December 2022

In [ ]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="xxxx",
)

def generate_response(query):
    # Retrieve relevant documents
    relevant_docs = retrieve_relevant_documents(query)
    print("Retrieved documents Found")
    
    if not relevant_docs:
        return "I'm sorry, this information is not available in the resume."
    
    # Compute similarity per segment
    similarity, best_doc = compute_similarity_per_segment(query, relevant_docs)

    # Use the most relevant document as context
    context = "\n".join(relevant_docs)  # Combine documents into a single string
    prompt = (
        "You are 'Chotu', Aneesh's assistant, answering questions about a person's resume. "
        "Use only the provided context to answer the question. "
        "Do not generate any information that is not explicitly stated in the context. "
        "Dont Explicitly Mention the name of the Context. "
        "If the information is not available, respond with: 'I'm sorry, this information is not available.'\n\n"
        f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    )
    
    try:
        # Call the OpenRouter API
        print("Calling OpenRouter API")
        completion = client.chat.completions.create(
            model="mistralai/mistral-7b-instruct:free",  # You can use "gpt-4" or other models supported on OpenRouter.
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a helpful assistant that answers questions about resumes based "
                        "only on the provided context."
                    )
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )
        
        # Extract the response content
        answer = completion.choices[0].message.content.strip()
        return answer

    except Exception as e:
        print(f"Error: {e}")
        return "An error occurred while processing your request."

# Example query
query = "Can Aneesh python ?"
print(generate_response(query))


Retrieved documents Found
Calling OpenRouter API
Yes, Aneesh has Python listed as one of his technical skills.
